In [122]:
import requests
import boto3
import json
import uuid
import random



from datetime import timezone
import datetime


In [200]:
def get_utc_seconds():
    dt = datetime.datetime.now(timezone.utc)
  
    utc_time = dt.replace(tzinfo=timezone.utc)
    return round(utc_time.timestamp()*1000)

class Optio:
    # dev.optio.cloud:5000/api/ui
    def __init__(self, username, password, instance='dev.optio.cloud'):
        self.__instance = instance
        
        self.__url = 'http://{}:5000/api/'.format(instance)
        self.__access_token = self.authenticate(username, password, app_client_id='110868d55nq4qtqm2kp62curm9')
        
        maps = self.request('GET', 'site_maps')
        self.curr_map_id = maps[0]['_id']
        
    @property
    def access_token(self):
        return self.__access_token
        
    def authenticate(self, username, password, app_client_id):
        client = boto3.client('cognito-idp', region_name='us-east-2')
    
        resp = client.initiate_auth(
            # UserPoolId=user_pool_id,
            ClientId=app_client_id,
            AuthFlow='USER_PASSWORD_AUTH',
            AuthParameters={
                "USERNAME": username,
                "PASSWORD": password
            }
        )
        
        print('Login success')
        return resp['AuthenticationResult']['IdToken']
        
    def request(self, method='GET', operator='', payload=None):

        if operator in ['stations', 'processes', 'dashboards', 'tasks', 'lot_templates', 'cards'] and method == 'GET':
            operator = 'site_maps/{}/{}'.format(self.curr_map_id, operator)
        
        response = requests.request(
            method,
            url=self.__url + operator,
            headers={
                'Authorization': '{}'.format(self.__access_token),
                'Access-Control-Allow-Origin': '*', 
                'Content-Type': 'application/json'     
            },
            data=json.dumps(payload))
        
        if method != 'DELETE':
            try:
                return json.loads(response.json())
            except Exception:
                raise Exception(response.text)
        
        
    ## ========== Basic GET functions ========= ##
    
    # ----- GET by ID ----- #
    def get_station_by_id(self, ID):
        return self.request('GET', 'stations/{}'.format(ID))
    
    def get_route_by_id(self, ID):
        return self.request('GET', 'tasks/{}'.format(ID))
    
    def get_process_by_id(self, ID):
        return self.request('GET', 'processes/{}'.format(ID))
    
    def get_lot_by_id(self, ID):
        return self.request('GET', 'cards/{}'.format(ID))
    
    def get_lot_template_by_id(self, ID):
        return self.request('GET', 'lot_template/{}'.format(ID))
    
    def get_dashboard_by_id(self, ID):
        return self.request('GET', 'dashboards/{}'.format(ID))
    
    def get_map_by_id(self,
                      ID):
        return self.request('GET', 'site_maps/{}'.format(ID))
    
    # ----- GET by Name ----- #
    def get_station_by_name(self, name):
        stations = self.request('GET', 'stations')
        return next(station for station in stations if station['name'] == name)
    
    def get_route_by_name(self, name):
        routes = self.request('GET', 'tasks')
        return next(route for route in routes if route['name'] == name)
    
    def get_process_by_name(self, name):
        processes = self.request('GET', 'processes')
        return next(process for process in processes if process['name'] == name)
    
    def get_lot_by_name(self, name):
        lots = self.request('GET', 'cards')
        return next(lot for lot in lots if lot['name'] == name)
    
    def get_lot_template_by_name(self, process_id, name):
        #lot_templates = self.request('GET', 'lot_templates')
        lot_templates=self.request('GET','site_maps/{}/cards/templates'.format(self.curr_map_id))

        return [lot_template for lot_template in lot_templates if lot_template['name'] == name and lot_template['processId']==process_id]
    
    def get_map_by_name(self, name):
        site_maps = self.request('GET', 'site_maps')
        return next(site_map for site_map in site_maps if site_map['name'] == name)
    
    # ---- GET Collections ----- #
    
    def get_processes(self):
        return self.request('GET','processes')
    
    def get_stations(self):
        return self.request('GET','stations')
    
    def get_all_lot_ids(self):
            lots = self.request('GET','/site_maps/{}/cards'.format(self.curr_map_id))
            return [lot['_id'] for lot in lots]
        
    def get_all_routes(self):
        return self.request('GET', 'tasks')

    
    
    
    ## ========== Basic Set function =========== ##
        
    def delete_lot_by_id(self, ID):
        return self.request('DELETE', 'cards/{}'.format(ID))
    
#   return self.request('GET', 'cards/{}'.format(ID))

    
    def delete_all_lots(self):
        for _id in self.get_all_lot_ids():
            self.delete_lot_by_id(_id)
            
    ## ========== Sim Helper Functions ========= ##
    def get_sources_from_process(self,process):
        sources=[]
        routes=process['routes']
        load_stations = [self.get_route_by_id(route_id)['load'] for route_id in routes]
        unload_stations = [self.get_route_by_id(route_id)['unload'] for route_id in routes]
        for ls in load_stations:
            #name=optio.get_station_by_id(ls)['name']   
            if(ls not in unload_stations):
                #print('{} is a source'.format(name))
                sources.append(ls)
        return sources


    def find_route_between_stations(self, process_name, load_station_name, unload_station_name):
        routes = self.request('GET', 'tasks')
        process = self.get_process_by_name(process_name)
        if not process: raise Exception('Process {} does not exist'.format(process_name))
        for route in routes:
            load_station = self.get_station_by_id(route['load'])
            unload_station = self.get_station_by_id(route['unload'])
            
            if route['processId'] == process['_id'] and load_station['name'] == load_station_name and unload_station['name'] == unload_station_name:
                return route
            
        raise Exception('Route between {} and {} does not exist'.format(load_station_name, unload_station_name))
        
    def find_lots_at_station(self, station_name):
        station_id = self.get_station_by_name(station_name)['_id']
        cards = self.request('GET', 'cards')
        cards_at_station = [card for card in cards if station_id in card['bins'].keys()]
        return cards_at_station
    
    def move_lot(self, lot_id, quantity, route_id, log_event=True, operator='Operator'):
        
        # Find lot by ID
        lot = self.get_lot_by_id(lot_id)
        if not lot: raise Exception('Lot not found')
        
        # Find route by ID
        route = self.get_route_by_id(route_id)
        if not route: raise Exception('Route not found')
        
        load_station = self.get_station_by_id(route['load'])
        load_dashboard = self.get_dashboard_by_id(load_station['dashboards'][0])
        
        # Extract load and unload stations from route
        load_station_id = route['load']
        unload_station_id = route['unload']
        
        # Bins contain the information about how many parts are at what station
        # Make sure this move event is valid
        bins = lot['bins']
        if load_station_id not in bins.keys():
            raise Exception('There are no parts of lot at route load station')
        elif bins[load_station_id]['count'] < quantity:
            raise Exception('Cannot move {} parts from lot when only {} parts exist at load station'.format(quantity, bins[load_station_id]['count']))
        
        # Set the new quantity at the unload station
        if unload_station_id in bins.keys():
            bins[unload_station_id]['count'] += quantity
        else:
            bins[unload_station_id] = {
                'count': quantity
            }
        
        # Subtract the move quantity from the load station, and delete that bin if it is now empty
        bins[load_station_id]['count'] -= quantity
        if bins[load_station_id]['count'] <= 0 and len(bins[load_station_id].keys()) == 1:
            del bins[load_station_id]
        
        # Update the lot in the DB
        lot['bins'] = bins  
        self.request('PUT', 'cards/{}'.format(lot['_id']), lot)
        
        # Log touch event
        if log_event:
            utc_seconds = get_utc_seconds()
            touch_event = {
                "dashboard_id": load_dashboard['_id']['$oid'],
                "load_station_id": route['load'],
                "lot_id": lot_id,
                "notes": "",
                "product_group_id": lot['lotTemplateId'],
                "quantity": quantity,
                "route_id": route_id,
                "sku": "default",
                "start_time": utc_seconds,
                "stop_time": utc_seconds,
                "unload_station_id": route['unload'],
                "user": operator
                }
            
            self.request('POST', 'touch_events', touch_event)
            
            
    def create_lot(self, lot, log_event=True, operator='Operator'):
        
        self.request('POST', 'cards', lot)
        
        
    def create_lots(self, lots):
        for lot in lots:
            self.create_lot(lot)

    def kickoff_lot(self, lot_id, quantity, station_id, log_event=True, operator='Operator'):
        
        # Find lot by ID
        lot = self.get_lot_by_id(lot_id)
        if not lot: raise Exception('Lot not found')
        
        # Find station by ID
        station = self.get_station_by_id(station_id)
        if not station: raise Exception('Station not found')
        
        unload_station_id = station_id
        
        # Bins contain the information about how many parts are at what station
        # Make sure this move event is valid
        bins = lot['bins']
        if 'QUEUE' not in bins.keys():
            raise Exception('There are no parts of lot in the queue')
        elif bins['QUEUE']['count'] < quantity:
            raise Exception('Cannot move {} parts from lot when only {} parts exist at QUEUE'.format(quantity, bins['QUEUE']['count']))
        
        # Set the new quantity at the unload station
        if unload_station_id in bins.keys():
            bins[unload_station_id]['count'] += quantity
        else:
            bins[unload_station_id] = {
                'count': quantity
            }
        
        # Subtract the move quantity from the load station, and delete that bin if it is now empty
        bins['QUEUE']['count'] -= quantity
        if bins['QUEUE']['count'] <= 0 and len(bins['QUEUE'].keys()) == 1:
            del bins['QUEUE']
        
        # Update the lot in the DB
        lot['bins'] = bins  
        self.request('PUT', 'cards/{}'.format(lot['_id']), lot)

In [203]:
optio = Optio('nikolaus@optio.cloud', 'Vaygts99')

Login success


In [204]:
optio.curr_map_id = optio.get_map_by_name('Phunkshun')['_id']

In [205]:
optio.delete_all_lots()

In [193]:
template_id=optio.get_lot_template_by_name('Basic')['_id']
template_id

'6174a5a804497e973f19985b'

In [191]:
optio.request('DELETE','/cards/templates/6176ed857134ed0c53957539')

In [198]:
lot_templates=optio.request('GET','site_maps/{}/cards/templates'.format(optio.curr_map_id))
lot_templates

[{'_id': '6170ea6e1d78ead8cd59518c',
  'fields': [[{'_id': 'e11b4950-4e3a-4c27-bd74-57925b470755',
     'component': 'TEXT_BOX_BIG',
     'dataType': 'STRING',
     'fieldName': 'description',
     'required': False,
     'showInPreview': True,
     'key': 0}],
   [{'component': 'TEXT_BOX',
     'dataType': 'STRING',
     '_id': 'b045222f-e29e-41e2-a8b3-ed9acc2c8404',
     'fieldName': 'Binding type',
     'required': True,
     'showInPreview': True}],
   [{'component': 'INPUT_BOX',
     'dataType': 'STRING',
     '_id': '89816fcf-95af-4fdb-ae8e-109238414ec9',
     'fieldName': 'Tip angle',
     'required': False,
     'showInPreview': True}],
   [{'_id': '58900099-fd9c-459c-8e6a-4d04acc597a9',
     'component': 'CALENDAR_START_END',
     'dataType': 'DATE_RANGE',
     'fieldName': 'dates',
     'required': False,
     'showInPreview': True,
     'key': 1}]],
  'name': 'Snowboards',
  'displayNames': {'name': 'Name', 'count': 'Quantity'},
  'processId': '3d0a9c29-4d9b-499d-90f1-583291

In [176]:
lot=optio.get_lot_by_id(optio.get_all_lot_ids()[0])

In [154]:
station=optio.get_station_by_name('Plank cutting')
station_id=station['_id']
process=optio.get_process_by_name('Plank cutting')
name="Test"
map_id=optio.curr_map_id

In [194]:
               template={'_id':str(uuid.uuid4()),
                          'bins': {station_id :{'count':100}},
                           "fields": [[{'_id': 'DEFAULT_DESCRIPTION_FIELD_ID',
                                        'component': 'TEXT_BOX_BIG',
                                        'dataType': 'STRING',
                                        'fieldName': 'description',
                                        'required': False,
                                        'showInPreview': True,
                                        'key': 0,
                                        'value': ''}],
                                      [{'_id': 'DEFAULT_DATES_FIELD_ID',
                                        'component': 'CALENDAR_START_END',
                                        'dataType': 'DATE_RANGE',
                                        'fieldName': 'dates',
                                        'required': False,
                                        'showInPreview': True,
                                        'key': 1,
                                        'value': [None, None]}]],
                           "flags": [],
                          'lotNum' : 0,
                          'lotTemplateId' : template_id,
                          'name':name,
                          'description': "",
                          'process_id':process["_id"],
                          'map_id':map_id,
                          'totalQuantity': 1,
                          'syncWithTemplate' : False,
                          #'start_date' : {'year':2021, 'month':2, 'day':random.randrange(5,7)},
                          #'end_date' : None,
                          
                         }

In [196]:
a=optio.create_lot(template)

{'_id': '6176f69f6346c64db1d3e9af', 'bins': {'87101f65-519c-4e29-b0f2-fb10afa45c50': {'count': 100}}, 'fields': [[{'_id': 'DEFAULT_DESCRIPTION_FIELD_ID', 'component': 'TEXT_BOX_BIG', 'dataType': 'STRING', 'fieldName': 'description', 'required': False, 'showInPreview': True, 'key': 0, 'value': ''}], [{'_id': 'DEFAULT_DATES_FIELD_ID', 'component': 'CALENDAR_START_END', 'dataType': 'DATE_RANGE', 'fieldName': 'dates', 'required': False, 'showInPreview': True, 'key': 1, 'value': [None, None]}]], 'flags': [], 'lotNum': 0, 'lotTemplateId': '6174a5a804497e973f19985b', 'name': 'Test', 'description': '', 'process_id': 'f21719c2-6695-4f05-8915-4967e3c4bb89', 'map_id': '04356d73-3253-4114-a35b-1ed3eec6a17b', 'totalQuantity': 1, 'syncWithTemplate': False}


In [69]:
lot_id=optio.get_all_lot_ids()[0]
lot=optio.get_lot_by_id(lot_id)

In [75]:
lot['_id']={}
lot

{'_id': {},
 'name': '',
 'bins': {'87101f65-519c-4e29-b0f2-fb10afa45c50': {'count': 1}},
 'flags': [],
 'process_id': 'f21719c2-6695-4f05-8915-4967e3c4bb89',
 'lotTemplateId': '6174a5a804497e973f19985b',
 'lotNum': 387,
 'fields': [[{'_id': 'DEFAULT_DESCRIPTION_FIELD_ID',
    'component': 'TEXT_BOX_BIG',
    'dataType': 'STRING',
    'fieldName': 'description',
    'required': False,
    'showInPreview': True,
    'key': 0,
    'value': ''}],
  [{'_id': 'DEFAULT_DATES_FIELD_ID',
    'component': 'CALENDAR_START_END',
    'dataType': 'DATE_RANGE',
    'fieldName': 'dates',
    'required': False,
    'showInPreview': True,
    'key': 1,
    'value': [None, None]}]],
 'totalQuantity': 1,
 'syncWithTemplate': False,
 'map_id': '04356d73-3253-4114-a35b-1ed3eec6a17b'}

In [9]:
for proc in processes:
    print(proc['name'])

Snowboard
Plank cutting


In [ ]:
route = optio.find_route_between_stations('Snowboard', 'Mount binding', 'Quality Inspection')

In [ ]:
lots_at_printing = optio.find_lots_at_station('Queue')

In [ ]:
lots_at_printing[0]['_id']
#optio.delete_lot_by_id(lots_at_printing[0]['_id'])

In [ ]:
optio.move_lot(lots_at_printing[0]['_id'], 1, route['_id'])

In [ ]:
optio.get_station_by_name('Mount binding')

In [ ]:
maps = 